In [7]:
import pandas as pd
import numpy as np
from  graphviz import Source
from scipy import misc
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, hamming_loss
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
dataframe = pd.DataFrame()
for x in range(1,4):
    dataframe_name = 'data/block_'+str(x)+'.csv'
    new_dataframe = pd.read_csv(dataframe_name)
    dataframe = pd.concat([dataframe,new_dataframe])
frame = dataframe


In [10]:
broken_columns = list(frame.columns[2:11])

for column in broken_columns:
    frame[column] = frame[column].apply(lambda x: np.NaN if x == '?' else x)
    frame[column] = frame[column].apply(lambda x: float(x) if type(x) == str else x)


In [11]:
frame.drop(['cmp_fname_c2','cmp_lname_c2'],axis=1,inplace=True)

In [12]:
def preparer_data(frame):
    frame["cmp_fname_c1"] = frame["cmp_fname_c1"].replace(np.NaN,0.000235404896421846)
    frame["cmp_lname_c1"] = frame["cmp_lname_c1"].replace(np.NaN,2.68694413843136e-05)
    frame["cmp_sex"] = frame["cmp_sex"].replace(np.NaN,0.5)
    frame["cmp_bd"] = frame["cmp_bd"].replace(np.NaN,0.032258064516129)
    frame["cmp_bm"] = frame["cmp_bm"].replace(np.NaN,0.0833333333333333)
    frame["cmp_by"] = frame["cmp_by"].replace(np.NaN, 0.00943396226415094)
    frame["cmp_plz"] = frame["cmp_plz"].replace(np.NaN, 0.000422654268808115)
    
    return frame

frame = preparer_data(frame)

In [13]:
def make_target_frame(frame, target_column):
    df_mod = frame.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].map({True:1, False:0})
    return (df_mod, targets)
df2, targets = make_target_frame(frame, "is_match")
frame.head()


,id_1,id_2,cmp_fname_c1,cmp_lname_c1,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
0,37291,53113,0.833333,1.0,1,1.0,1.0,1.0,0.0,True
1,39086,47614,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
2,70031,70237,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
3,84795,97439,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
4,36950,42116,1.000000,1.0,1,1.0,1.0,1.0,1.0,True


In [14]:
columns = list(df2.columns[0:7])
y = df2["Target"]
x = frame[columns]
tree = DecisionTreeClassifier()
tree.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
def visualize_tree(tree, feature_names):
    
    with open("tree.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "tree.dot", "-o", "tree.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Não foi possivel abrir o arquivo")
    
visualize_tree(tree, columns)

<img src="tree.png">

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


In [17]:
tree.fit(x_train, y_train)
preds = tree.predict(x_test)

In [18]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
(tn, fp, fn, tp)

(567108, 22, 28, 2006)

In [19]:
print(accuracy_score(y_test, preds))


0.9999121518578125
